# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Lidia Fabra Cuenca  <br>
Url: https://github.com/lidiaf28/03MIAR-Algoritmos-de-optimizacion/blob/main/TrabajoPractico_LidiaFabra.ipynb<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. **Organizar los horarios de partidos de La Liga**<br>
>3. Combinar cifras y operaciones

## Descripción del problema:

Desde La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada.
Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos
a los horarios de forma que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes (horario 24h):

* Viernes: 20.

* Sábado: 12, 16, 18, y 20.

* Domingo: 12, 16, 18 y 20.

* Lunes: 20.

En primer lugar se clasifican los equipos en tres categorías según el número de seguidores (que tiene relación
directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en
horario de sábado a las 20h (el mejor en todos los casos):

* Categoría A-A: 2 millones.

* Categoría A-B: 1.3 millones.

* Categoría A-C: 1 millón.

* Categoría B-B: 0.9 millones.

* Categoría B-C: 0.75 millones.

* Categoría C-C: 0.47 millones.

Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes
siguientes (horarios de 24h):

* Viernes 20: 0.4

* Sábado 12: 0.55

* Sábado 16: 0.7

* Sábado 18: 0.8

* Sábado 20: 1

* Domingo 12: 0.45

* Domingo 16: 0.75

* Domingo 18: 0.85

* Domingo 20: 1

* Lunes 20: 0.4

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes.

Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se
estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

* 0 coincidencias: 0%

* 1 coincidencias: 25%

* 2 coincidencias: 45%

* 3 coincidencias: 60%

* 4 coincidencias: 70%

* 5 coincidencias: 75%

* 6 coincidencias: 78%

* 7 coincidencias: 80%

* 8 coincidencias: 80%

De esta forma se podrán realizar los cálculos necesarios para cada jornada.
            

### (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

Con los datos del enunciado se puede observar que hay 10 horarios disponibles, para los 10 partidos que debe haber (20 equipos, juegan 2 por partido) para cada jornada.

En este caso, cada uno de los 10 partidos puede jugarse en cualquiera de los 10 horarios disponibles. No se tiene en cuenta cuántos partidos pueden coincidir en el mismo horario, ni restricciones de audiencia o categorías.

Por lo tanto, el número total de combinaciones posibles sin restricciones es:

$10^{10}$


### ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?

Si tenemos en cuenta las restricciones, sabemos que si o si debe haber partidos el viernes y el lunes. Asignando un partido al viernes quedan 9 partidos, y luego otro al lunes, quedan 8.
El resto de cruces se repartiran en 10^7 combinaciones posibles.

Por tanto, el número total de combinaciones posibles con restricciones es (por jornada):

9 * 8 * $10^7$



## Modelo para el espacio de soluciones<br>
### (*)¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.


Se han utilizado diccionarios para organizar la información del problema de manera estructurada y eficiente. Los diccionarios permiten asociar claves con valores, facilitando el acceso rápido a datos específicos, como los coeficientes de horario y las audiencias por categorías de equipos. Esta estructura es ideal para consultar o actualizar la información de manera ágil.

Por otro lado, se ha empleado una lista para almacenar todos los horarios disponibles y gestionar la asignación de estos horarios a los partidos. También se ha utilizado una lista con las reducciones de coincidencias para ajustar la audiencia en función del número de partidos que coinciden en un horario. Al gestionar los índices de la lista, resulta sencillo buscar el coeficiente de reducción correspondiente al número de partidos repetidos. Además, se ha empleado una lista para los nombres de los equipos a repartir en los partidos.


### (*)¿Cual es la función objetivo? ¿Es un problema de maximización o minimización?

La función objetivo en este problema de optimización es maximizar la audiencia total de los partidos a lo largo de una jornada, teniendo en cuenta lo siguiente:

- Categoria equipo: Audiencia base según las categorías de los equipos que juegan el partido.

- Coeficiente Horario: Coeficiente que depende del horario en el que se juega el partido.

- Coincidencia partido: El porcentaje de reducción por coincidencia de partidos en el mismo horario.

- I(viernes y lunes) es un indicador que toma el valor 1 si hay un partido el viernes y otro el lunes, y 0 en caso contrario.

Por tanto, para el calculo de la audiencia se tendrá en cuenta la siguiente fórmula:

                Audiencia Total = Σ (audiencia_categoria * coef_horario * (1 - coincidencias)) * I(viernes y lunes)




### Diseña un algoritmo para resolver el problema por fuerza bruta

In [1]:
import pandas as pd
import numpy as np
import itertools
import random

In [2]:
horarios = [
    'viernes 20:00',
    'sabado 12:00',
    'sabado 16:00',
    'sabado 18:00',
    'sabado 20:00',
    'domingo 12:00',
    'domingo 16:00',
    'domingo 18:00',
    'domingo 20:00',
    'lunes 20:00'
]
audiencia_categoria = {
    ('A', 'A'): 2_000_000,
    ('A', 'B'): 1_300_000,
    ('A', 'C'): 1_000_000,
    ('B', 'B'): 900_000,
    ('B', 'C'): 750_000,
    ('C', 'C'): 470_000
}
coef_horario = {
    'viernes 20:00': 0.4,
    'sabado 12:00': 0.55,
    'sabado 16:00': 0.7,
    'sabado 18:00': 0.8,
    'sabado 20:00': 1,
    'domingo 12:00': 0.45,
    'domingo 16:00': 0.75,
    'domingo 18:00': 0.85,
    'domingo 20:00': 1,
    'lunes 20:00': 0.4
}

reduccion_coincidencias = [0, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80]

equipos = ['Valencia', 'Real Madrid', 'Barcelona', 'Atlético Madrid', 'Sevilla', 'Betis',
           'Athletic Club', 'Real Sociedad', 'Villarreal', 'Espanyol', 'Granada', 'Osasuna',
           'Levante', 'Mallorca', 'Cádiz', 'Almería', 'Rayo Vallecano', 'Elche', 'Getafe', 'Celta']

In [3]:
def generar_partidos(equipos): # Generar 10 partidos aleatorios sin repetir equipos
    equipos_disponibles = equipos.copy()  
    random.shuffle(equipos_disponibles)
    partidos = []

    while len(equipos_disponibles) >= 2:
        equipo1 = equipos_disponibles.pop(0)
        equipo2 = equipos_disponibles.pop(0) 
        partidos.append((equipo1, equipo2))

    return partidos

# Función objetivo
def calcular_audiencia(audiencia_categoria, coef_horario, num_coincidencias):
    coincidencia = reduccion_coincidencias[num_coincidencias]
    return audiencia_categoria * coef_horario * (1 - coincidencia)

def verificar_equipos(asignacion, partidos_jornada):
    equipos_asignados = set()
    for i, partido in enumerate(partidos_jornada):
        equipo_local, equipo_visitante = partido
        if equipo_local in equipos_asignados or equipo_visitante in equipos_asignados:
            return False
        equipos_asignados.add(equipo_local)
        equipos_asignados.add(equipo_visitante)
    return True


def calcular_audiencia_total(asignacion,partidos_jornada):
    audiencia_total = 0
    horario_count = {h: 0 for h in horarios}  # {clave: valor for item in iterador}
    if 'viernes 20:00' not in asignacion or 'lunes 20:00' not in asignacion:
        return 0  # Si falta un partido en esos días, la audiencia es 0
    for i, partido in enumerate(partidos_jornada):
        equipo1, equipo2 = partido
        categoria1 = 'A' if equipo1 in ['Valencia', 'Barcelona', 'Real Madrid'] else ('C' if equipo1 in ['Granada', 'Almería', 'Mallorca', 'Elche', 'Real Sociedad','Celta'] else 'B')
        categoria2 = 'A' if equipo2 in ['Valencia', 'Barcelona', 'Real Madrid'] else ('C' if equipo2 in ['Granada', 'Almería', 'Mallorca', 'Elche', 'Real Sociedad', 'Celta'] else 'B')

        #buscar en diccionario la audiencia segun categorias
        audiencia_base = audiencia_categoria.get((categoria1, categoria2), 0)
        horario = asignacion[i] #horario asignado
        coef = coef_horario[horario]#coeficiente segun horario
        num_coincidencias = horario_count[horario] #num partidos a la misma hora

        audiencia_total += calcular_audiencia(audiencia_base, coef, num_coincidencias)

        horario_count[horario] += 1 # Incremento de partidos en ese horario

    return audiencia_total

def generar_jornadas(k, equipos):
    jornadas = {}
    for jornada in range(k):
        partidos_jornada = generar_partidos(equipos)
        jornadas[jornada + 1] = partidos_jornada
    return jornadas

def fuerza_bruta(k):

    jornadas = generar_jornadas(k, equipos)
    max_audiencia = 0
    mejor_asignacion_dict = {}
    max_audiencia_total = 0

    for jornada, partidos_jornada in jornadas.items():
        max_audiencia = 0
        mejor_asignacion = None

        # Buscar la mejor asignación para la jornada actual
        for asignacion in itertools.permutations(horarios, len(partidos_jornada)):
            if verificar_equipos(asignacion,partidos_jornada):
                audiencia_total = calcular_audiencia_total(asignacion,partidos_jornada)

                if audiencia_total > max_audiencia:
                    max_audiencia = audiencia_total
                    mejor_asignacion = asignacion
        # Diccionario que asocia cada partido con su horario
        mejor_asignacion_dict[jornada] = {partido: horario for partido, horario in zip(partidos_jornada, mejor_asignacion)} #{clave: valor for elemento in iterador}
        max_audiencia_total += max_audiencia

    return mejor_asignacion_dict, max_audiencia_total

# Ejecución del algoritmo por fuerza bruta
k=2
mejor_asignacion, max_audiencia = fuerza_bruta(k)

print("Mejor asignación de horarios y partidos:")

for jornada, asignacion in mejor_asignacion.items():
    print(f"\nJornada {jornada}:")
    for partido, horario in asignacion.items():
        print(f"{partido}: {horario}")

print(f"\nMáxima audiencia total para {k} jornadas: {max_audiencia}")


Mejor asignación de horarios y partidos:

Jornada 1:
('Real Sociedad', 'Mallorca'): sabado 12:00
('Elche', 'Sevilla'): viernes 20:00
('Celta', 'Getafe'): domingo 12:00
('Almería', 'Athletic Club'): lunes 20:00
('Valencia', 'Osasuna'): sabado 20:00
('Barcelona', 'Granada'): domingo 18:00
('Real Madrid', 'Levante'): domingo 20:00
('Espanyol', 'Cádiz'): sabado 16:00
('Villarreal', 'Betis'): sabado 18:00
('Rayo Vallecano', 'Atlético Madrid'): domingo 16:00

Jornada 2:
('Levante', 'Sevilla'): sabado 16:00
('Real Sociedad', 'Granada'): viernes 20:00
('Valencia', 'Athletic Club'): sabado 20:00
('Elche', 'Osasuna'): lunes 20:00
('Espanyol', 'Atlético Madrid'): sabado 18:00
('Barcelona', 'Getafe'): domingo 20:00
('Cádiz', 'Mallorca'): sabado 12:00
('Real Madrid', 'Almería'): domingo 18:00
('Betis', 'Rayo Vallecano'): domingo 16:00
('Villarreal', 'Celta'): domingo 12:00

Máxima audiencia total para 2 jornadas: 12146500.0


### Calcula la complejidad del algoritmo por fuerza bruta

Para cada permutación, se recorren todos los partidos. Si tienes n partidos y n horarios, el número total de formas en que puedes asignar esos horarios a los partidos es n!

Por ejemplo: Para 5 partidos seria
5! = 120 asignaciones posibles.
Para 10 partidos:
10! = 3,628,800 asignaciones posibles.

Si consideramos que se puede tener un número distinto de horarios, la situación puede variar, pero generalmente, al usar todos los horarios disponibles, las permutaciones se mantendrán en n!. Si tenemos en cuenta k jornadas, se necesita evaluar todas las combinaciones posibles de horarios en cada jornada. Esto significa que la complejidad se multiplica por k, lo que da una complejidad total de O(k*n!) Este tipo de complejidad es bastante alta y se vuelve inviable para valores grandes de n y k.

### (*) Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Una opción para mejorar el algoritmo de fuerza bruta es emplear técnicas heurísticas para reducir la complejidad del problema. Aunque una técnica determinista podría garantizar la solución óptima al problema, el coste computacional asociado a este enfoque aumenta cuando el número de partidos o jornadas crece. Para este tipo de problemas, donde el espacio de búsqueda crece exponencialmente, una técnica determinista resulta ineficaz para problemas de mayor escala, como cuando consideramos múltiples jornadas de liga.

En este caso, se propone una solución mediante Búsqueda Local. Esta técnica comienza con una asignación inicial de horarios y realiza pequeños cambios iterativos para mejorarla. De este modo, se puede empezar con una asignación de horarios aleatoria y luego iterar para intercambiar horarios entre partidos con el objetivo de maximizar la audiencia total.

Este enfoque proporciona una forma práctica y efectiva de encontrar una solución cercana a la óptima, evitando el examen exhaustivo de todas las combinaciones posibles y ofreciendo una solución escalable cuando el problema crece en tamaño.

In [4]:
def generar_vecino(asignacion):
    # nueva solución vecina intercambiando dos horarios de la asignación actual.
    nuevo_asignacion = asignacion[:]
    i, j = random.sample(range(len(asignacion)), 2) # 2 indices aleatorios
    nuevo_asignacion[i], nuevo_asignacion[j] = nuevo_asignacion[j], nuevo_asignacion[i]
    return nuevo_asignacion

def busqueda_local(k):
    mejor_asignacion_dict = {}
    max_audiencia_total = 0

    for jornada in range(1, k + 1):
        partidos_jornada = generar_partidos(equipos)
        mejor_asignacion = random.sample(horarios, len(partidos_jornada))
        mejor_audiencia = calcular_audiencia_total(mejor_asignacion,partidos_jornada)

        while True:
            mejorado = False
            for _ in range(len(partidos_jornada)):
                vecino = generar_vecino(mejor_asignacion)
                audiencia_vecino = calcular_audiencia_total(vecino,partidos_jornada)

                if audiencia_vecino > mejor_audiencia:
                    mejor_asignacion = vecino
                    mejor_audiencia = audiencia_vecino
                    mejorado = True

            if not mejorado:  # no hay mejoras, se sale
                break

        # Diccionario que asocia cada partido con su horario
        mejor_asignacion_dict[jornada] = {partido: horario for partido, horario in zip(partidos_jornada, mejor_asignacion)}
        max_audiencia_total += mejor_audiencia

    return mejor_asignacion_dict, max_audiencia_total

# Ejecución del algoritmo de búsqueda local
k = 2
mejor_asignacion, mejor_audiencia = busqueda_local(k)

print("Mejor asignación de horarios y partidos:")

for jornada, asignacion in mejor_asignacion.items():
    print(f"\nJornada {jornada}:")
    for partido, horario in asignacion.items():
        print(f"{partido}: {horario}")

print(f"\nMáxima audiencia total para {k} jornadas: {mejor_audiencia}")

Mejor asignación de horarios y partidos:

Jornada 1:
('Atlético Madrid', 'Villarreal'): sabado 18:00
('Getafe', 'Sevilla'): domingo 16:00
('Celta', 'Granada'): viernes 20:00
('Almería', 'Elche'): sabado 16:00
('Real Sociedad', 'Valencia'): lunes 20:00
('Athletic Club', 'Rayo Vallecano'): domingo 18:00
('Espanyol', 'Levante'): sabado 20:00
('Betis', 'Osasuna'): sabado 12:00
('Real Madrid', 'Mallorca'): domingo 20:00
('Cádiz', 'Barcelona'): domingo 12:00

Jornada 2:
('Getafe', 'Athletic Club'): sabado 18:00
('Espanyol', 'Granada'): domingo 16:00
('Real Madrid', 'Barcelona'): domingo 20:00
('Almería', 'Elche'): viernes 20:00
('Mallorca', 'Betis'): sabado 12:00
('Celta', 'Cádiz'): sabado 16:00
('Sevilla', 'Atlético Madrid'): domingo 18:00
('Real Sociedad', 'Villarreal'): lunes 20:00
('Osasuna', 'Valencia'): domingo 12:00
('Rayo Vallecano', 'Levante'): sabado 20:00

Máxima audiencia total para 2 jornadas: 10207500.0


### (*)Calcula la complejidad del algoritmo

Con este algoritmo, la complejidad se reduce a O(k*n*m), donde:

- n es el número de partidos.

- m es el número de iteraciones hasta encontrar una solución localmente óptima

- k es el numero de jornadas

  
Con esta complejidad se obtiene una escala linealmente tanto con n como con k, lo que la hace más eficiente y adecuada para escenarios de mayor tamaño.

De esta manera, se busca una solución cercana a la óptima evitando el cálculo exhaustivo de todas las permutaciones posibles. En lugar de explorar todas las posibles asignaciones de horarios, la búsqueda local se centra en mejorar una solución existente realizando pequeños cambios en la asignación actual.
Esto hace que la búsqueda local sea significativamente más eficiente en términos de tiempo, especialmente para problemas de tamaño relativamente grande, ya que no necesita evaluar todas las permutaciones posibles. Aunque no garantiza encontrar la mejor solución global, proporciona una solución buena en un tiempo mucho más reducido, lo que lo convierte en una opción práctica y efectiva para problemas complejos.

### Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Se va a diseñar un juego de datos aleatorios variando los coeficientes de audiencia y categorías de los equipos, y con el numero de jornadas aleatorio.

In [5]:

coef_horario = {
    'viernes 20:00': round(random.uniform(0.3, 0.5), 2),
    'sabado 12:00': round(random.uniform(0.5, 0.6), 2),
    'sabado 16:00': round(random.uniform(0.6, 0.8), 2),
    'sabado 18:00': round(random.uniform(0.7, 0.9), 2),
    'sabado 20:00': 1,  #horario de máxima audiencia segun enunciado
    'domingo 12:00': round(random.uniform(0.4, 0.5), 2),
    'domingo 16:00': round(random.uniform(0.7, 0.8), 2),
    'domingo 18:00': round(random.uniform(0.8, 0.9), 2),
    'domingo 20:00': 1, #horario de máxima audiencia segun enunciado
    'lunes 20:00': round(random.uniform(0.3, 0.5), 2)
}

audiencia_categoria_aleatoria = {
    ('A', 'A'): random.randint(1_800_000, 2_200_000), # rango valores más altos para los partidos A
    ('A', 'B'): random.randint(1_200_000, 1_400_000),
    ('A', 'C'): random.randint(900_000, 1_100_000),
    ('B', 'B'): random.randint(800_000, 1_000_000),
    ('B', 'C'): random.randint(600_000, 800_000),
    ('C', 'C'): random.randint(400_000, 500_000)
}

### Aplica el algoritmo al juego de datos generado

In [6]:
k = random.randint(1, 10)
mejor_asignacion, mejor_audiencia = busqueda_local(k)

print(f"\nNúmero de jornadas: {k}")
print("Mejor asignación de horarios y partidos (con datos aleatorios):")

for jornada, asignacion in mejor_asignacion.items():
    print(f"\nJornada {jornada}:")
    for partido, horario in asignacion.items():
        print(f"{partido}: {horario}")

print(f"\nMáxima audiencia total: {mejor_audiencia}")


Número de jornadas: 7
Mejor asignación de horarios y partidos (con datos aleatorios):

Jornada 1:
('Espanyol', 'Elche'): sabado 12:00
('Sevilla', 'Getafe'): domingo 18:00
('Athletic Club', 'Real Sociedad'): lunes 20:00
('Villarreal', 'Cádiz'): sabado 16:00
('Osasuna', 'Almería'): viernes 20:00
('Atlético Madrid', 'Celta'): sabado 18:00
('Betis', 'Barcelona'): domingo 12:00
('Rayo Vallecano', 'Levante'): domingo 16:00
('Valencia', 'Granada'): domingo 20:00
('Real Madrid', 'Mallorca'): sabado 20:00

Jornada 2:
('Getafe', 'Rayo Vallecano'): domingo 20:00
('Villarreal', 'Mallorca'): sabado 16:00
('Espanyol', 'Atlético Madrid'): domingo 16:00
('Elche', 'Athletic Club'): lunes 20:00
('Levante', 'Valencia'): domingo 12:00
('Barcelona', 'Osasuna'): sabado 20:00
('Celta', 'Real Madrid'): viernes 20:00
('Granada', 'Almería'): sabado 12:00
('Sevilla', 'Cádiz'): domingo 18:00
('Betis', 'Real Sociedad'): sabado 18:00

Jornada 3:
('Barcelona', 'Rayo Vallecano'): sabado 20:00
('Celta', 'Granada'): v

### Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Como posible avance en el estudio del problema, se podría explorar la implementacion de algoritmos genéticos como otra solución de optimización.Estos algoritmos podrian ofrecer soluciones mas eficientes a medida que se incrementara el numero de partidos y jornadas.Además, sería interesante analizar el impacto de variaciones en los coeficientes de audiencia y las categorías de equipos en los resultados, así como considerar la incorporación de restricciones adicionales, como las preferencias de los equipos o limitaciones logísticas, para hacer el modelo más realista y aplicable a situaciones del mundo real